In [43]:
from azureml.core import Workspace, Experiment, Run, Model

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to work with', ws.name)

2023-03-19:00:19:06,144 INFO     [workspace.py:291] Found the config file in: /config.json


Ready to work with dsba6190ml


In [44]:
## Get experiment and run of interest
experiment = Experiment(workspace=ws, name='instructor_automl_diabetes')
run = Run(experiment, run_id = "AutoML_2f266fb1-d22a-41f4-9ec2-58ee52e1ae69_46")

In [45]:
## Register the model
print('Registering model...')
run.register_model(model_path='outputs/model.pkl', model_name='inst_automl_diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC Weighed': run.get_metrics()['AUC_weighted'],
                               'Weighted Accuracy': run.get_metrics()['weighted_accuracy']})

## Get the registered model
model = ws.models['inst_automl_diabetes_model']

print('Model registered.')

Registering model...
Model registered.


## Deploy a model as a web service

In [46]:
import os

## Create a folder for the deployment files
deployment_folder = './diabetes_service'
os.makedirs(deployment_folder, exist_ok=True)
print(deployment_folder, 'folder created.')

## Set path for scoring script
script_file = 'score_diabetes.py'
script_path = os.path.join(deployment_folder,script_file)

./diabetes_service folder created.


Now you need an entry script that the service will use to score new data.

In [47]:
%%writefile $script_path
import json
import joblib
import numpy as np
import os
import pandas as pd

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = json.loads(raw_data)['data']

    df = pd.DataFrame(data,
                      columns = [
                          "PatientID",
                          "Pregnancies",
                          "PlasmaGlucose",
                          "DiastolicBloodPressure",
                          "TricepsThickness",
                          "SerumInsulin",
                          "BMI",
                          "DiabetesPedigree",
                          "Age"
                      ] 
                     )
    
    # Get a prediction from the model
    predictions = model.predict(df)
    
    # print the data and predictions (so they'll be logged!)
    log_text = 'Data:' + str(data) + ' - Predictions:' + str(predictions)
    print(log_text)
    
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

Overwriting ./diabetes_service/score_diabetes.py


Now you can deploy the service (in this case, as an Azure Container Instance (ACI).

> **Note**: This can take a few minutes - wait until the state is shown as **Healthy**.

In [48]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

## Configure the scoring environment
## Curated environments: https://learn.microsoft.com/en-us/azure/machine-learning/resource-curated-environments
curated_env_name = 'AzureML-AutoML'
aci_service_env = Environment.get(workspace=ws, name=curated_env_name)

# aci_service_env = Environment(name='inst_automl_aci_service-env')
# python_packages = ['scikit-learn', 'azureml-defaults', 'azureml-sdk[automl]', 'azure-ml-api-sdk']
# for package in python_packages:
#     aci_service_env.python.conda_dependencies.add_pip_package(package)

inference_config = InferenceConfig(source_directory=deployment_folder,
                                   entry_script=script_file,
                                   environment=aci_service_env)

## Configure the web service container
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

2023-03-19:00:19:21,601 WARNING  [model.py:2242] Warning, custom base image or base dockerfile detected without a specified `inferencing_stack_version`. Please set environment.inferencing_stack_version='latest'


In [49]:
## Deploy the model as a service

print('Deploying model...')
service_name = "inst-automl-diabetes-service1"
aci_service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
aci_service.wait_for_deployment(show_output = True)
print(aci_service.state)

Deploying model...
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-03-19 00:19:25+00:00 Creating Container Registry if not exists.
2023-03-19 00:19:25+00:00 Registering the environment.
2023-03-19 00:19:26+00:00 Use the existing image.
2023-03-19 00:19:27+00:00 Submitting deployment to compute.
2023-03-19 00:19:29+00:00 Checking the status of deployment inst-automl-diabetes-service1..
2023-03-19 00:21:32+00:00 Checking the status of inference endpoint inst-automl-diabetes-service1.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


## Use the web service

With the service deployed, now you can consume it from a client application.

First, determine the URL to which these applications must submit their requests.

In [50]:
endpoint = aci_service.scoring_uri
print(endpoint)

http://d0280e8b-eef6-48ec-849a-46003f4a7d95.eastus.azurecontainer.io/score


Now that you know the endpoint URI, an application can simply make an HTTP request, sending the patient data in JSON (or binary) format, and receive back the predicted class(es).

> **Tip**: If an error occurs because the service endpoint isn't ready. Wait a few seconds and try again!

In [58]:
import requests
import json

# Create new data for inferencing
# x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
#          [0,148,58,11,179,39.19207553,0.160829008,45]]

x_new = {"data": [
    [1, 2,180,74,24,21,23.9091702,1.488172308,22],
    [2, 0,148,58,11,179,39.19207553,0.160829008,45],
    [1218879, 1,124,82,42,266,34.98577243,0.083335019,25],
    [1940297, 2,44,81,46,146,34.53408234,0.693502171,55]
]}

# Convert the array to a serializable list in a JSON document
input_json = json.dumps(x_new)

# Set the content type
headers = { 'Content-Type':'application/json' }

# Get the predictions
predictions = requests.post(endpoint, input_json, headers = headers)
print(predictions.status_code)

if predictions.status_code == 200:
    predicted_classes = json.loads(predictions.json())
    for i in range(len(x_new['data'])):
        print ("Patient {}".format(x_new['data'][i]), predicted_classes[i] )

200
Patient [1, 2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22] diabetic
Patient [2, 0, 148, 58, 11, 179, 39.19207553, 0.160829008, 45] not-diabetic
Patient [1218879, 1, 124, 82, 42, 266, 34.98577243, 0.083335019, 25] not-diabetic
Patient [1940297, 2, 44, 81, 46, 146, 34.53408234, 0.693502171, 55] diabetic
